In [1]:
import opentrons.execute
from opentrons import protocol_api
from opentrons import types

In [2]:
protocol = opentrons.execute.get_protocol_api('2.13')
protocol.home()

In [ ]:
import opentrons.execute
from opentrons import protocol_api
from opentrons import types

metadata = {
    'protocolName': 'MEGAA reaction on Opentron OT-2',
    'author': 'Yiming Huang',
    'description': 'Simple protocol to get started using OT-2',
    'apiLevel': '2.13'
}

## 8 samples with same templates and different mutagenesis oligo pool, 3 rounds of MEGAA reaction will be performed

def run(protocol: protocol_api.ProtocolContext):

    # load labware
    magnetic_module = protocol.load_module('magnetic module gen2', location = '4')
    temperature_module = protocol.load_module('temperature module', location = '3')
    thermocycler_module = protocol.load_module('Thermocycler Module')
    
    reagent_strip = temperature_module.load_labware('opentrons_96_aluminumblock_generic_pcr_strip_200ul')
    PCR_plate = thermocycler_module.load_labware('nest_96_wellplate_100ul_pcr_full_skirt')
    magnetic_plate = magnetic_module.load_labware('nest_96_wellplate_100ul_pcr_full_skirt')
    SPRI_reservoir = protocol.load_labware('nest_12_reservoir_15ml', location = '1')
    
    pipette_P300 = protocol.load_instrument('p300_multi_gen2', 'right', tip_racks = [tiprack_right])
    pipette_P20 = protocol.load_instrument('p20_multi_gen2', 'left', tip_racks = [tiprack_right])
    
    
    # total number of input samples: 8
    input_samples = 8
    
    ############################################
    ########### start of the protocol ##########
    ############################################
 
    if temperature_module.lid_position != 'open':
        temperature_module.open_lid()

    ## disengage magnetic module and set temperatures for temperature module and thermocycler module.
    magnetic_module.disengage()
    temperature_module.set_temperature(4)
    thermocycler_module.set_block_temperature(12)
    
    
    ################################################
    ########### round-1 of MEGAA reaction ##########
    ################################################
    
    
    ## transfer 0.5uL DNA template from reagent strip (column-1) to PCR plate (column-1)
    pipette_P20.pick_up_tip()
    pipette_P20.transfer(0.5,
                          reagent_strip.columns_by_name()['1'],
                          PCR_plate.columns_by_name()['1'],
                          new_tip = 'never')
    pipette_P20.drop_tip()
    
    
    ## transfer 49.5uL Utemplate-generation master mix from reagent strip (column-6)
    ## to PCR plate (column-1) and mix well by pipetting
    pipette_P300.transfer(49.5,
                          reagent_strip.columns_by_name()['6'],
                          PCR_plate.columns_by_name()['1'],
                          new_tip = 'always', mix_after = (5, 35),
                          trash = True, home_after = False)

    
    ## run PCR to generate U-containing template for MEGAA reaction;
    ## annealing temperature is set to be 60°C and can be changed according to primer design.
    ## extenstion time is set to be 60 seconds and can be changed according to the length of desired product.
    annealing_temp = 60
    extension_time = 60
    run_thermocycler(thermocycler_module, annealing_temp, extension_time)
    
    if thermocycler_module.lid_position != 'open':
        thermocycler_module.open_lid()
        
    
    ## trasnfer 19.6uL of nuclease free water from SPRI reservoir (column-1) to magnetic plate (column-1)
    ## no need to change tips since all of them are water
    pipette_P20.pick_up_tip()
    pipette_P20.transfer(19.6,
                          SPRI_reservoir.columns_by_name()['1'],
                          magnetic_plate.columns_by_name()['1'],
                          new_tip = 'never')
    pipette_P20.drop_tip(home_after = False)
    
    
    ## transfer 0.4uL U-containing template from PCR plate (column-1) to magnetic plate (column-1)
    ## and mix well by pipetting 
    pipette_P20.transfer(0.4,
                          PCR_plate.columns_by_name()['1'],
                          magnetic_plate.columns_by_name()['1'],
                          new_tip = 'always', mix_after = (3, 10),
                          trash = True, home_after = False)
    
    
    ## perform SPRI beads cleanup with 1.5X ratio, U-containing template will be eluted into 80uL of nuclease free water
    input_columns = ['1']
    input_volume = 20
    ratio_of_SPRI_cleanup = 1.5
    volume_of_elution = 80
    do_SPRI_beads_cleanup(input_columns, input_volume, pipette_P20, pipette_P300, \
                          magnetic_plate, SPRI_reservoir, ratio_of_SPRI_cleanup, volume_of_elution)
    
    
    ## transfer 2.8uL of MEGAA mutagenesis reaction master mix from reagent strip (column-8) to PCR plate (column-3)
    pipette_P20.pick_up_tip()
    pipette_P20.transfer(2.8,
                          reagent_strip.columns_by_name()['8'],
                          PCR_plate.columns_by_name()['3'],
                          new_tip = 'never')
    pipette_P20.drop_tip()
    
    ## transfer 1uL of MEGAA mutagenesis oligo pools from reagent strip (column-2) to PCR plate (column-3)
    pipette_P20.transfer(1,
                         reagent_strip.columns_by_name()['2'],
                         PCR_plate.columns_by_name()['3'],
                         new_tip = 'always', trash = True, home_after = False)
    
    ## transfer 0.2uL of U-containing template from magnetic plate (column-1) to PCR plate (column-3)
    ## and mix well by pipetting
    pipette_P20.transfer(0.2,
                         magnetic_plate.columns_by_name()['1'],
                         PCR_plate.columns_by_name()['3'],
                         new_tip = 'always', mix_after = (5, 2),
                         trash = True, home_after = False)
    

    ## run MEGAA reaction on thermocycler
    run_MEGAA_reaction(thermocycler_module)
    if temperature_module.lid_position != 'open':
        temperature_module.open_lid()

        
    ## transfer 46uL MEGAA amplification master mix from reagent strip (column-7) to PCR plate (column-3)
    ## and mix well by pipetting
    pipette_P300.transfer(46,
                          reagent_strip.columns_by_name()['7'],
                          PCR_plate.columns_by_name()['3'],
                          new_tip = 'always', mix_after = (5, 35),
                          trash = True, home_after = False)
    
    
 
    ## run PCR to amplify product from MEGAA mutagenesis reaction
    ## annealing temperature is set to be 60°C and can be changed according to primer design.
    ## extenstion time is set to be 60 seconds and can be changed according to the length of desired product.
    annealing_temp = 60
    extension_time = 60
    run_thermocycler(thermocycler_module, annealing_temp, extension_time)
 
    
    ################################################
    ########### round-2 of MEGAA reaction ##########
    ################################################
    
    
    ## transfer 0.4uL of round-1 MEGAA product from PCR plate (column-3) to magnetic plate (column-2)
    ## add 79.6uL of nuclease-free water to dilute the product 200-fold
    pipette_P20.transfer(0.4,
                         PCR_plate.columns_by_name()['3'],
                         magnetic_plate.columns_by_name()['2'],
                         new_tip = 'always', trash = True, home_after = False)
    pipette_P300.transfer(79.6,
                          SPRI_reservoir.columns_by_name()['1'],
                          magnetic_plate.columns_by_name()['2'],
                          mix_after = (10, 50),
                          new_tip = 'always', trash = True, home_after = False)
    
    ## transfer 0.4uL of 1:200 diluted round-1 MEGAA product from magnetic plate (column-2) to magnetic plate (column-3)
    ## add  79.6uL of nuclease-free water to further dilute the product, so 1:40,000 dilution in total
    pipette_P20.transfer(0.4,
                         magnetic_plate.columns_by_name()['2'],
                         magnetic_plate.columns_by_name()['3'],
                         new_tip = 'always', trash = True, home_after = False)
    pipette_P300.transfer(79.6,
                          SPRI_reservoir.columns_by_name()['1'],
                          magnetic_plate.columns_by_name()['3'],
                          mix_after = (10, 50),
                          new_tip = 'always', trash = True, home_after = False)
    
    
    ## transfer 0.5uL diluted round-1 MEGAA product from magnetic plate (column-3) to PCR plate (column-5)
    pipette_P20.pick_up_tip()
    pipette_P20.transfer(0.5,
                          magnetic_plate.columns_by_name()['3'],
                          PCR_plate.columns_by_name()['5'],
                          new_tip = 'never')
    pipette_P20.drop_tip()
    
    
    ## transfer 49.5uL Utemplate-generation master mix from reagent strip (column-6)
    ## to PCR plate (column-5) and mix well by pipetting
    pipette_P300.transfer(49.5,
                          reagent_strip.columns_by_name()['6'],
                          PCR_plate.columns_by_name()['5'],
                          new_tip = 'always', mix_after = (5, 35),
                          trash = True, home_after = False)

    
    ## run PCR to generate U-containing template for MEGAA reaction;
    ## annealing temperature is set to be 60°C and can be changed according to primer design.
    ## extenstion time is set to be 60 seconds and can be changed according to the length of desired product.
    annealing_temp = 60
    extension_time = 60
    run_thermocycler(thermocycler_module, annealing_temp, extension_time)
    
    if thermocycler_module.lid_position != 'open':
        thermocycler_module.open_lid()
        
    
    ## trasnfer 19.6uL of nuclease free water from SPRI reservoir (column-1) to magnetic plate (column-4)
    ## no need to change tips since all of them are water
    pipette_P20.pick_up_tip()
    pipette_P20.transfer(19.6,
                          SPRI_reservoir.columns_by_name()['1'],
                          magnetic_plate.columns_by_name()['4'],
                          new_tip = 'never')
    pipette_P20.drop_tip(home_after = False)
    
    
    ## transfer 0.4uL U-containing template from PCR plate (column-5) to magnetic plate (column-4)
    ## and mix well by pipetting 
    pipette_P20.transfer(0.4,
                          PCR_plate.columns_by_name()['5'],
                          magnetic_plate.columns_by_name()['4'],
                          new_tip = 'always', mix_after = (3, 10),
                          trash = True, home_after = False)
    
    
    ## perform SPRI beads cleanup with 1.5X ratio, U-containing template will be eluted into 80uL of nuclease free water
    input_columns = ['4']
    input_volume = 20
    ratio_of_SPRI_cleanup = 1.5
    volume_of_elution = 80
    do_SPRI_beads_cleanup(input_columns, input_volume, pipette_P20, pipette_P300, \
                          magnetic_plate, SPRI_reservoir, ratio_of_SPRI_cleanup, volume_of_elution)
    
    
    ## transfer 2.8uL of MEGAA mutagenesis reaction master mix from reagent strip (column-8) to PCR plate (column-7)
    pipette_P20.pick_up_tip()
    pipette_P20.transfer(2.8,
                          reagent_strip.columns_by_name()['8'],
                          PCR_plate.columns_by_name()['7'],
                          new_tip = 'never')
    pipette_P20.drop_tip()
    
    ## transfer 1uL of MEGAA mutagenesis oligo pools from reagent strip (column-2) to PCR plate (column-7)
    pipette_P20.transfer(1,
                         reagent_strip.columns_by_name()['2'],
                         PCR_plate.columns_by_name()['7'],
                         new_tip = 'always', trash = True, home_after = False)
    
    ## transfer 0.2uL of U-containing template from magnetic plate (column-4) to PCR plate (column-7)
    ## and mix well by pipetting
    pipette_P20.transfer(0.2,
                         magnetic_plate.columns_by_name()['4'],
                         PCR_plate.columns_by_name()['7'],
                         new_tip = 'always', mix_after = (5, 2),
                         trash = True, home_after = False)
    

    ## run MEGAA reaction on thermocycler
    run_MEGAA_reaction(thermocycler_module)
    if temperature_module.lid_position != 'open':
        temperature_module.open_lid()

        
    ## transfer 46uL MEGAA amplification master mix from reagent strip (column-7) to PCR plate (column-7)
    ## and mix well by pipetting
    pipette_P300.transfer(46,
                          reagent_strip.columns_by_name()['7'],
                          PCR_plate.columns_by_name()['7'],
                          new_tip = 'always', mix_after = (5, 35),
                          trash = True, home_after = False)
    
    ## run PCR to amplify product from MEGAA mutagenesis reaction
    ## annealing temperature is set to be 60°C and can be changed according to primer design.
    ## extenstion time is set to be 60 seconds and can be changed according to the length of desired product.
    annealing_temp = 60
    extension_time = 60
    run_thermocycler(thermocycler_module, annealing_temp, extension_time)

    
    
    ################################################
    ########### round-3 of MEGAA reaction ##########
    ################################################
    
    
    ## transfer 0.4uL of round-2 MEGAA product from PCR plate (column-7) to magnetic plate (column-5)
    ## add 79.6uL of nuclease-free water to dilute the product 200-fold
    pipette_P20.transfer(0.4,
                         PCR_plate.columns_by_name()['7'],
                         magnetic_plate.columns_by_name()['5'],
                         new_tip = 'always', trash = True, home_after = False)
    pipette_P300.transfer(79.6,
                          SPRI_reservoir.columns_by_name()['1'],
                          magnetic_plate.columns_by_name()['5'],
                          mix_after = (10, 50),
                          new_tip = 'always', trash = True, home_after = False)
    
    ## transfer 0.4uL of 1:200 diluted round-1 MEGAA product from magnetic plate (column-5) to magnetic plate (column-6)
    ## add  79.6uL of nuclease-free water to further dilute the product, so 1:40,000 dilution in total
    pipette_P20.transfer(0.4,
                         magnetic_plate.columns_by_name()['5'],
                         magnetic_plate.columns_by_name()['6'],
                         new_tip = 'always', trash = True, home_after = False)
    pipette_P300.transfer(79.6,
                          SPRI_reservoir.columns_by_name()['1'],
                          magnetic_plate.columns_by_name()['6'],
                          mix_after = (10, 50),
                          new_tip = 'always', trash = True, home_after = False)
    
    
    ## transfer 0.5uL diluted round-2 MEGAA product from magnetic plate (column-6) to PCR plate (column-9)
    pipette_P20.pick_up_tip()
    pipette_P20.transfer(0.5,
                          magnetic_plate.columns_by_name()['6'],
                          PCR_plate.columns_by_name()['9'],
                          new_tip = 'never')
    pipette_P20.drop_tip()
    
    
    ## transfer 49.5uL Utemplate-generation master mix from reagent strip (column-6)
    ## to PCR plate (column-9) and mix well by pipetting
    pipette_P300.transfer(49.5,
                          reagent_strip.columns_by_name()['6'],
                          PCR_plate.columns_by_name()['9'],
                          new_tip = 'always', mix_after = (5, 35),
                          trash = True, home_after = False)

    
    ## run PCR to generate U-containing template for MEGAA reaction;
    ## annealing temperature is set to be 60°C and can be changed according to primer design.
    ## extenstion time is set to be 60 seconds and can be changed according to the length of desired product.
    annealing_temp = 60
    extension_time = 60
    run_thermocycler(thermocycler_module, annealing_temp, extension_time)
    
    if thermocycler_module.lid_position != 'open':
        thermocycler_module.open_lid()
        
    
    ## trasnfer 19.6uL of nuclease free water from SPRI reservoir (column-1) to magnetic plate (column-7)
    ## no need to change tips since all of them are water
    pipette_P20.pick_up_tip()
    pipette_P20.transfer(19.6,
                          SPRI_reservoir.columns_by_name()['1'],
                          magnetic_plate.columns_by_name()['7'],
                          new_tip = 'never')
    pipette_P20.drop_tip(home_after = False)
    
    
    ## transfer 0.4uL U-containing template from PCR plate (column-9) to magnetic plate (column-7)
    ## and mix well by pipetting 
    pipette_P20.transfer(0.4,
                          PCR_plate.columns_by_name()['9'],
                          magnetic_plate.columns_by_name()['7'],
                          new_tip = 'always', mix_after = (3, 10),
                          trash = True, home_after = False)
    
    
    ## perform SPRI beads cleanup with 1.5X ratio, U-containing template will be eluted into 80uL of nuclease free water
    input_columns = ['7']
    input_volume = 20
    ratio_of_SPRI_cleanup = 1.5
    volume_of_elution = 80
    do_SPRI_beads_cleanup(input_columns, input_volume, pipette_P20, pipette_P300, \
                          magnetic_plate, SPRI_reservoir, ratio_of_SPRI_cleanup, volume_of_elution)
    
    
    ## transfer 2.8uL of MEGAA mutagenesis reaction master mix from reagent strip (column-8) to PCR plate (column-11)
    pipette_P20.pick_up_tip()
    pipette_P20.transfer(2.8,
                          reagent_strip.columns_by_name()['8'],
                          PCR_plate.columns_by_name()['11'],
                          new_tip = 'never')
    pipette_P20.drop_tip()
    
    ## transfer 1uL of MEGAA mutagenesis oligo pools from reagent strip (column-2) to PCR plate (column-11)
    pipette_P20.transfer(1,
                         reagent_strip.columns_by_name()['2'],
                         PCR_plate.columns_by_name()['11'],
                         new_tip = 'always', trash = True, home_after = False)
    
    ## transfer 0.2uL of U-containing template from magnetic plate (column-7) to PCR plate (column-11)
    ## and mix well by pipetting
    pipette_P20.transfer(0.2,
                         magnetic_plate.columns_by_name()['7'],
                         PCR_plate.columns_by_name()['11'],
                         new_tip = 'always', mix_after = (5, 2),
                         trash = True, home_after = False)
    

    ## run MEGAA reaction on thermocycler
    run_MEGAA_reaction(thermocycler_module)
    if temperature_module.lid_position != 'open':
        temperature_module.open_lid()

        
    ## transfer 46uL MEGAA amplification master mix from reagent strip (column-7) to PCR plate (column-11)
    ## and mix well by pipetting
    pipette_P300.transfer(46,
                          reagent_strip.columns_by_name()['7'],
                          PCR_plate.columns_by_name()['11'],
                          new_tip = 'always', mix_after = (5, 35),
                          trash = True, home_after = False)
    
    ## run PCR to amplify product from MEGAA mutagenesis reaction
    ## annealing temperature is set to be 60°C and can be changed according to primer design.
    ## extenstion time is set to be 60 seconds and can be changed according to the length of desired product.
    annealing_temp = 60
    extension_time = 60
    run_thermocycler(thermocycler_module, annealing_temp, extension_time)

    
    
    ############################################
    ########### final product cleanup ##########
    ############################################
    
    
    ## transfer 20uL final MEGAA product from PCR plate (column-11) to magnetic plate (column-11)
    pipette_P20.transfer(20,
                          PCR_plate.columns_by_name()['11'],
                          magnetic_plate.columns_by_name()['11'],
                          new_tip = 'always', trash = True, home_after = False)
    
    
    ## perform SPRI beads cleanup with 1.5X ratio, final product will be eluted into 55uL of nuclease free water
    input_columns = ['11']
    input_volume = 20
    ratio_of_SPRI_cleanup = 1.5
    volume_of_elution = 55
    do_SPRI_beads_cleanup(input_columns, input_volume, pipette_P20, pipette_P300, \
                          magnetic_plate, SPRI_reservoir, ratio_of_SPRI_cleanup, volume_of_elution)

    
    ## transfer 50uL of final product from magentic plate (column-11) to reagent strip (column-12)
    pipette_P300.transfer(50,  
                          magnetic_plate.columns_by_name()['11'],
                          reagent_strip.columns_by_name()['12'],
                          new_tip = 'always', trash = True, home_after = False)
 
    ##########################################
    ########### end of the protocol ##########
    ##########################################
 
    

def run_thermocycler(thermocycler_module, annealing_temp, extension_time):

    total_volume = 50
    lid_temp = 105
    PCR_cycles = 30
    initial_temp = 98
    initial_time = 30
    denaturation_temp = 98
    denaturation_time = 10
    annealing_time = 10
    extension_temp = 72
    final_temp = 72
    final_time = 60 * 5
    hold_temp = 12
    
    if thermocycler_module.lid_position != 'closed':
        thermocycler_module.close_lid()

    thermocycler_module.set_lid_temperature(lid_temp)

    # initialization
    thermocycler_module.set_block_temperature(initial_temp, hold_time_seconds = initial_time,
                                 block_max_volume = PCR_volume)

    # run profile
    profile = [
        {'temperature': denaturation_temp, 'hold_time_seconds': denaturation_time},
        {'temperature': annealing_temp, 'hold_time_seconds': annealing_time},
        {'temperature': extension_temp, 'hold_time_seconds': extension_time}
    ]

    thermocycler_module.execute_profile(steps = profile, repetitions = PCR_cycles, block_max_volume = PCR_volume)

    # final elongation
    thermocycler_module.set_block_temperature(final_temp, hold_time_seconds = final_time,
                                 block_max_volume = PCR_volume)
    
    # final hold
    thermocycler_module.deactivate_lid()
    thermocycler_module.set_block_temperature(hold_temp)
    

def run_MEGAA_reaction(thermocycler_module):
    reaction_volume = 4
    lid_temp = 105
    hold_temp = 12
    
    if thermocycler_module.lid_position != 'closed':
        thermocycler_module.close_lid()

    thermocycler_module.set_lid_temperature(lid_temp)
    
    profile = [
        {'temperature': 95, 'hold_time_seconds': 90},
        {'temperature': 4, 'hold_time_seconds': 60},
        {'temperature': 50, 'hold_time_seconds': 60},
        {'temperature': 55, 'hold_time_seconds': 60},
        {'temperature': 60, 'hold_time_seconds': 60},
        {'temperature': 65, 'hold_time_seconds': 60 * 90}
    ]
    
    thermocycler_module.execute_profile(steps = profile, repetitions = 1, block_max_volume = reaction_volume)
    
    # final hold
    thermocycler_module.deactivate_lid()
    thermocycler_module.set_block_temperature(hold_temp)

    
def do_SPRI_beads_cleanup(input_columns, input_volume, pipette_P20, pipette_P300, \
                          magnetic_plate, SPRI_reservoir, ratio_of_SPRI_cleanup, volume_of_elution):
   
 
    volume_of_EtOH_wash = 80
   
    time_DNA_binding_min = 2
    time_beads_binding_min = 9
    time_EtOH_wash_sec = 30
    time_EtOH_dry_min = 3
    time_water_elution_min = 2


    pipette_P300.transfer(ratio_of_SPRI_cleanup * input_volume,
                          SPRI_reservoir.columns_by_name()['2'],
                          magnetic_plate.columns_by_name()input_columns,
                          new_tip = 'always', mix_after = (20, ((ratio_of_SPRI_cleanup + 1) * input_volume) * 0.7),
                          trash = True, home_after = False)
    
    protocol.delay(minutes = time_DNA_binding_min)
    magnetic_module.engage()
    protocol.delay(minutes = time_beads_binding_min)
    
    for eachCol in input_columns:
        pipette_P300.pick_up_tip()
        pipette_P300.move_to(magnetic_plate[eachCol])
        pipette_P300.aspirate((ratio_of_SPRI_cleanup + 1) * input_volume * 1.2, rate = 0.3)
        pipette_P300.drop_tip(home_after = False)
    
    for eachCol in input_columns:
        pipette_P300.pick_up_tip()
        pipette_P300.transfer(volume_of_EtOH_wash,
                          SPRI_reservoir.columns_by_name()['3'],
                          magnetic_plate.columns_by_name()[eachCol],
                          new_tip = 'never', trash = False, home_after = False)
        protocol.delay(seconds = time_EtOH_wash_sec)
        pipette_P300.move_to(magnetic_plate[eachCol])
        pipette_P300.aspirate(volume_of_EtOH_wash * 1.2, rate = 0.3)
        pipette_P300.drop_tip(home_after = False)
        
    for eachCol in input_columns:
        pipette_P300.pick_up_tip()
        pipette_P300.transfer(volume_of_EtOH_wash,
                          SPRI_reservoir.columns_by_name()['3'],
                          magnetic_plate.columns_by_name()[eachCol],
                          new_tip = 'never', trash = False, home_after = False)
        protocol.delay(seconds = time_EtOH_wash_sec)
        pipette_P300.move_to(magnetic_plate[eachCol])
        pipette_P300.aspirate(volume_of_EtOH_wash * 1.2, rate = 0.3)
        pipette_P300.drop_tip(home_after = False)
  
    for eachCol in input_columns:
        pipette_P20.pick_up_tip()
        pipette_P20.move_to(magnetic_plate[eachCol])
        pipette_P20.aspirate(20, rate = 0.3)
        pipette_P20.drop_tip(home_after = False)
        
    magnetic_module.disengage()
    protocol.delay(minutes = time_EtOH_dry_min)
    
    pipette_P200.transfer(volume_of_elution,
                          SPRI_reservoir.columns_by_name()['1'],
                          magnetic_plate.columns_by_name()input_columns,
                          new_tip = 'always', mix_after = (20, volume_of_Utemplate_elution * 0.7),
                          trash = True, home_after = False)
    
    protocol.delay(minutes = time_water_elution_min)
    
    magnetic_module.engage()
    
    protocol.delay(minutes = time_beads_binding_min)
    


In [6]:
protocol = opentrons.execute.get_protocol_api('2.11')
protocol.home()
run(protocol)
protocol.home()